# imports

In [1]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from bestmodel import *
from cashAlgorithm.smacClass import ProblemType
from cashAlgorithm.Models import SARIMAModel

In [2]:
# # url = "https://www.openml.org/data/get_csv/31/dataset_31.csv"
# url='data.csv'
# df = pd.read_csv(url)
# #export the data to a csv file
# # df.to_csv('data.csv', index=False)
# df = df.dropna()

# # Convert categorical features to numerical
# df = pd.get_dummies(df, drop_first=True)
# target = "class_good"
# # X = df.drop(columns=target)
# y = df[target]

# X_train_raw, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# def Which_features(X_train,y_train,number_of_columns):

#     # Select the top k features based on ANOVA F-statistic
#     selector = SelectKBest(score_func=f_classif, k=number_of_columns)
#     X_train_selected = selector.fit_transform(X_train, y_train)

#     # Get the column names of the selected features
#     selected_feature_names = X_train.columns[selector.get_support()]

    
#     return list(selected_feature_names)

# selected_features = Which_features(X_train_raw,y_train,number_of_columns=15)
# X =X[selected_features]
# X_train_raw = X_train_raw[selected_features]
# X_test = X_test[selected_features]
# # selected_features


In [3]:
# #try svc with linear kernel and on datasetabove
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix
# svcobj = SVC(kernel='linear')
# svcobj.fit(X_train_raw, y_train)
# y_pred = svcobj.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)

In [4]:

# import warnings
# warnings.filterwarnings("ignore")
# #testsvm, test regression, test time series
# if __name__ == '__main__':
#     problemtype = ProblemType.CLASSIFICATION
#     choosenModels=['KNN','LR']
#     Bestmodelobj = Bestmodel(problemtype,choosenModels,X_train_raw,X_test,y_train,y_test)
#     Bestmodelobj.splitTestData()
#     Bestmodelobj.TrainModel()
#     predictions=Bestmodelobj.PredictModel(X_test)
#     print(Bestmodelobj.modelobj)


In [5]:
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# from sklearn import datasets
# iris = datasets.load_iris()

# X = iris.data[:, 1:]
# y = iris.data[:, 0] 

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# model = LinearRegression()
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# print("Mean Squared Error:", mse)

In [6]:
# # ['LinearRegression','Lasso','Ridge','RF','XGboost']
# if __name__ == "__main__":
#     problemtype = ProblemType.REGRESSION
#     choosenModels=['LR','Lasso','Ridge','RF','XGboost']
#     Bestmodelobj = Bestmodel(problemtype,choosenModels,X_train,X_test,y_train,y_test)
#     Bestmodelobj.splitTestData()
#     Bestmodelobj.TrainModel()
#     predictions=Bestmodelobj.PredictModel(X_test)
#     print(Bestmodelobj.modelobj)

In [7]:
from similaritySearch.functions import *
from bestmodel import *
from cashAlgorithm.smacClass import ProblemType
import pandas as pd


def calculate_date_frequency(series):
    """
    Calculate the most common frequency (interval) of a datetime series.

    Parameters:
    series (pd.Series): A pandas Series of datetime objects.

    Returns:
    str: A string representing the most common frequency (e.g., 'D' for days, 'H' for hours).
    """
    # Drop NaN values to avoid errors in calculations
    series = series.dropna()

    # Calculate the differences between consecutive dates
    diffs = series.diff().dropna()

    # Calculate the most common frequency
    freq = diffs.mode()[0]

    # Convert the frequency to a string representation
    series = series.dropna()

    # Calculate the differences between consecutive dates
    diffs = series.diff().dropna()

    # Calculate the most common frequency
    freq = diffs.mode()[0]

    # Convert the frequency to a string representation
    if freq == pd.Timedelta(days=1):
        return 'D'  # Daily
    elif freq >= pd.Timedelta(days=7):
        return 'W'  # Weekly
    elif freq >= pd.Timedelta(days=30):
        return 'M'  # Monthly
    elif freq >= pd.Timedelta(days=90):
        return 'Q'  # Quartely
    elif freq >= pd.Timedelta(days=365):
        return 'A'  # Annually
    else:
        return freq  # Return the exact frequency if it's not a common one


def Detections_(df, y_column, problem, date_col=None):
    if date_col:
        df[date_col] = pd.to_datetime(df[date_col])

    if problem == "timeseries":
        df.rename(columns={date_col: 'ds', y_column: 'y'}, inplace=True)
        y_column = 'y'
        df['y'] = df['y'].str.replace('[^0-9\.]', '', regex=True)
        df['y'] = pd.to_numeric(df['y'], errors='coerce')
        df['y'] = df['y'].astype(float)

    df = RemoveIDColumn.remove_high_cardinality_columns(df)
    df = MissingValues().del_high_null_cols(df)
    categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

    nulls_columns = MissingValues().detect_nulls(df)
    cols_with_outliers = Outliers().detect_outliers(df)
    df = Duplicates().handle_dub(df)
    imbalance_detected = None
    if problem == "classification":
        imbalance_detected = HandlingImbalanceClasses().detect_class_imbalance(df, y_column)
    df_without_y = df.drop(columns=[y_column])
    low_variance_columns, low_variance_info = HandlingColinearity().detect_low_variance(df_without_y)

    return df, nulls_columns, cols_with_outliers, imbalance_detected, low_variance_columns, categorical_columns


def _process_data(df: pd.DataFrame, fill_na_dict: dict, outliers_methods_input: tuple,
                  Norm_method: str) -> pd.DataFrame:
    """Handle missing values, outliers, normalization, and encoding."""
    df = MissingValues().handle_nan(df, fill_na_dict)
    cols_with_outliers = Outliers().detect_outliers(df)
    df = Outliers().handle_outliers(df, cols_with_outliers, outliers_methods_input)
    df = DataNormalization().normalize_data(df, Norm_method)

    return df


def Cleaning(df, problem, y_column, fill_na_dict, outliers_methods_input, imb_instruction, Norm_method,
             lowvariance_actions, encoding_dict, date_col=None):
    if problem == "timeseries":
        date_col = 'ds'
        y_column = 'y'
        frequency = calculate_date_frequency(df[date_col])
        df.set_index('ds', inplace=True)
        train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
    elif problem == "classification":
        train_data, test_data = train_test_split(df, test_size=0.2, random_state=42, stratify=df[y_column])
        train_data = train_data.reset_index(drop=True)
        test_data = test_data.reset_index(drop=True)
    else:
        df['timestamp'] = df[date_col].astype('int64') / 10 ** 9  # Convert to UNIX timestamp in seconds
        df['year'] = df[date_col].dt.year
        df['month'] = df[date_col].dt.month
        df['day'] = df[date_col].dt.day
        df.drop(date_col, axis=1, inplace=True)
        train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
        train_data = train_data.reset_index(drop=True)
        test_data = test_data.reset_index(drop=True)

    train_data = _process_data(train_data, fill_na_dict, outliers_methods_input, Norm_method)
    test_data = _process_data(test_data, fill_na_dict, outliers_methods_input, Norm_method)

    df_copy = pd.concat([train_data, test_data])
    df_copy = df_copy.sort_index()
    original_columns = set(df.columns)

    # Remove co-linearity & low_variance
    historical_df_without_y = df_copy.drop(columns=[y_column])
    historical_df_without_y = HandlingColinearity().handle_low_variance(historical_df_without_y, lowvariance_actions)
    historical_df_without_y = HandlingColinearity().handling_colinearity(historical_df_without_y)
    historical_df_without_y[y_column] = df_copy[y_column]

    # Update historical_df_copy with processed columns
    df_copy = historical_df_without_y.copy()

    # Identify removed columns
    processed_columns = set(df_copy.columns)
    removed_columns = list(original_columns - processed_columns)

    # Remove identified columns from train_data and test_data
    train_data = train_data.drop(columns=removed_columns)
    test_data = test_data.drop(columns=removed_columns)
    # if problem != "timeseries":
    # not needed now
    # meta_extractor, meta_features, best_models = extract_and_search_features(df_copy)

    train_data = EncodeCategorical().Encode(train_data, encoding_dict)
    test_data = EncodeCategorical().Encode(test_data, encoding_dict)

    if imb_instruction:
        train_data = HandlingImbalanceClasses().handle_class_imbalance(train_data, y_column, imb_instruction)
        test_data = HandlingImbalanceClasses().handle_class_imbalance(test_data, y_column, imb_instruction)

    df_copy = pd.concat([train_data, test_data])
    df_copy = df_copy.sort_index()

    if problem != "timeseries":
        x_train = train_data.drop(columns=[y_column])
        y_train = train_data[y_column]

        # Split test_data into features and labels
        x_test = test_data.drop(columns=[y_column])
        y_test = test_data[y_column]

        return x_train, y_train, x_test, y_test,
    else:
        return train_data, test_data, frequency


def user_interaction(df, problem, y_column, date_col=None):
    try:
        df, nulls_columns, cols_with_outliers, imbalance, low_variance_columns, categorical_columns = Detections_(df,
                                                                                                                  y_column,
                                                                                                                  problem,
                                                                                                                  date_col)

        # Handling missing values
        fill_na_dict = {}
        if nulls_columns:
            fill_na_dict = {col: 'auto' for col in nulls_columns}

        # Handling outliers
        outliers_method_input = ('z_score', 'auto', 3)
        imb_instruction = "auto" if imbalance else None
        Norm_method = "auto"
        low_actions = {}
        encoding_dict = {}
        if categorical_columns:
            encoding_dict = {col: 'auto' for col in categorical_columns}
        if low_variance_columns:
            low_actions = {col: 'auto' for col in low_variance_columns}

        if problem != "timeseries":
            x_train, y_train, x_test, y_test = Cleaning(df, problem, y_column, fill_na_dict, outliers_method_input,
                                                        imb_instruction, Norm_method, low_actions, encoding_dict,
                                                        date_col)
            return x_train, y_train, x_test, y_test

        else:
            train_data, test_data, frequency = Cleaning(df, problem, y_column, fill_na_dict, outliers_method_input,
                                                        imb_instruction, Norm_method, low_actions, encoding_dict,
                                                        date_col)
            return train_data, test_data, frequency

    except ValueError as ve:
        print(f"Error occurred: {ve}")
        return None


In [8]:
df1 =pd.read_csv(r"daily-minimum-temperatures-in-me.csv")
problemtype1 = "timeseries"
train_data, test_data,frequency = user_interaction(df1, problemtype1, "Daily minimum temperatures", date_col="Date")
choosenModels=["Arima"]
traindatax='lol'
test_datax='loll'
# print(test_data)
Bestmodelobj = Bestmodel(ProblemType.TIME_SERIES,choosenModels,traindatax,traindatax,train_data,test_data,frequency)
Bestmodelobj.splitTestData()
# print(Bestmodelobj.y_test1)
# Bestmodelobj.Getincumbent()
Bestmodelobj.TrainModel()


[INFO][abstract_initial_design.py:95] Reducing the number of initial configurations from 40 to 2 (max_ratio == 0.25).
[INFO][abstract_initial_design.py:147] Using 2 initial design configurations and 0 additional configurations.
let's start the optimization
[WARNING][abstract_runner.py:134] Target function returned infinity or nothing at all. Result is treated as CRASHED and cost is set to inf.
[INFO][abstract_intensifier.py:515] Added config 0963f6 as new incumbent because there are no incumbents yet.
[WARNING][abstract_runner.py:134] Target function returned infinity or nothing at all. Result is treated as CRASHED and cost is set to inf.


KeyboardInterrupt: 

In [ ]:
df2 = pd.read_csv(r"train.csv")
problemtype2 = "classification"
choosenModels = ["KNN", "LR", "RF"]
x_train, y_train, x_test, y_test = user_interaction(df2, problemtype2, "Survived", date_col=None)
Bestmodelobj = Bestmodel(ProblemType.CLASSIFICATION, choosenModels, x_train,x_test,y_train,y_test)
Bestmodelobj.splitTestData()
# Bestmodelobj.Getincumbent()
Bestmodelobj.TrainModel()

[INFO][abstract_initial_design.py:95] Reducing the number of initial configurations from 40 to 25 (max_ratio == 0.25).
[INFO][abstract_initial_design.py:147] Using 24 initial design configurations and 0 additional configurations.
let's start the optimization
[INFO][abstract_intensifier.py:305] Using only one seed for deterministic scenario.
[INFO][abstract_intensifier.py:515] Added config 61f4fc as new incumbent because there are no incumbents yet.
[INFO][abstract_intensifier.py:594] Added config 7d1035 and rejected config 61f4fc as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:594] Added config 3f9fd5 and rejected config 7d1035 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:594] Added config d53524 and rejected config 3f9fd5 as incumbent because it is not better than the incumbents on 1 instances:
The incumbent is: {'Models': 'LR', 'regularizationStre': 0.07983951867558062

In [ ]:
df3 = pd.read_csv(r"pulsedata.csv")
problemtype3 = "regression"
x_train, y_train, x_test, y_test = user_interaction(df3, problemtype3, "Calories", date_col="Date")
choosenModels = ['LinearRegression', "Lasso"]
Bestmodelobj = Bestmodel(ProblemType.REGRESSION, choosenModels, x_train,x_test,y_train,y_test)
Bestmodelobj.splitTestData()
# Bestmodelobj.Getincumbent()
Bestmodelobj.TrainModel()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Alaa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Alaa\AppData\Local\Temp\ipykernel_60348\2759679421.py", line 3, in <module>
    x_train, y_train, x_test, y_test = user_interaction(df3, problemtype3, "Calories", date_col="Date")
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Alaa\AppData\Local\Temp\ipykernel_60348\3285653845.py", line 162, in user_interaction
    df, nulls_columns, cols_with_outliers, imbalance, low_variance_columns, categorical_columns = Detections_(df,
                                                                                                  ^^^^^^^^^^^^^^^
  File "C:\Users\Alaa\AppData\Local\Temp\ipykernel_60348\3285653845.py", li

In [ ]:
path = r"daily-minimum-temperatures-in-me.csv"
df =pd.read_csv(path)
df.rename(columns={'Date': 'ds', 'Daily minimum temperatures': 'y'}, inplace=True)
df['ds'] = pd.to_datetime(df['ds'], format='%m/%d/%Y')
df['y'] = df['y'].str.replace('[^0-9\.]', '', regex=True)
df['y'] = pd.to_numeric(df['y'], errors='coerce')
df['y'] = df['y'].astype(float)
df.set_index('ds', inplace=True)
split_date = pd.to_datetime('1990-12-15')
train_data = df[df.index <= split_date]
test_data = df[df.index > split_date]
traindatax='lol'
test_datax='loll'

In [ ]:
train_data

,y
ds,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8
...,...
1990-12-11,11.1
1990-12-12,14.0
1990-12-13,11.4


In [ ]:
# from Models import SARIMAModel
# sarimasmac(train, test, p, q, d, P, Q, D, s, freq='D')
# results=SARIMAModel.Sarimasmac(train_data, test_data, 2, 0, 0, 3, 1, 3, 30, freq='D')
# resul/ts

In [ ]:
if __name__=="__main__":
    problemtype = ProblemType.TIME_SERIES
    choosenModels=['Sarima']
    Bestmodelobj = Bestmodel(problemtype,choosenModels,traindatax,traindatax,train_data,test_data)
    Bestmodelobj.splitTestData()
    # Bestmodelobj.Getincumbent()
    Bestmodelobj.TrainModel()
    

[INFO][abstract_initial_design.py:95] Reducing the number of initial configurations from 80 to 2 (max_ratio == 0.25).
[INFO][abstract_initial_design.py:147] Using 2 initial design configurations and 0 additional configurations.
let's start the optimization
[INFO][abstract_intensifier.py:305] Using only one seed for deterministic scenario.
[WARNING][abstract_runner.py:134] Target function returned infinity or nothing at all. Result is treated as CRASHED and cost is set to inf.
[INFO][abstract_intensifier.py:515] Added config 31b9e5 as new incumbent because there are no incumbents yet.
[INFO][abstract_intensifier.py:594] Added config 04a5f9 and rejected config 31b9e5 as incumbent because it is not better than the incumbents on 1 instances:
[WARNING][abstract_runner.py:134] Target function returned infinity or nothing at all. Result is treated as CRASHED and cost is set to inf.
[WARNING][abstract_runner.py:134] Target function returned infinity or nothing at all. Result is treated as CRAS

In [ ]:
predictions=Bestmodelobj.PredictModel(test_data)

In [ ]:
predictions

1990-12-16    13.043275
1990-12-17     9.459163
1990-12-18    11.359163
1990-12-19    10.559163
1990-12-20     8.959163
1990-12-21    15.659163
1990-12-22    12.859163
1990-12-23    11.159163
1990-12-24    11.759163
1990-12-25    13.659163
1990-12-26    13.559163
1990-12-27     9.459163
1990-12-28    15.359163
1990-12-29    15.359163
1990-12-30    13.359163
1990-12-31    13.959163
Freq: D, Name: predicted_mean, dtype: float64